In [1]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


In [2]:
# Paths to the dataset
train_dir = '/kaggle/input/deep-learning-practice-image-classification/train'
test_dir = '/kaggle/input/deep-learning-practice-image-classification/test'

In [3]:
# List the train folders
print(os.listdir(train_dir))

# Check a specific folder
print(os.listdir(os.path.join(train_dir, 'Amphibia')))


['Reptilia', 'Animalia', 'Arachnida', 'Amphibia', 'Aves', 'Mollusca', 'Fungi', 'Insecta', 'Plantae', 'Mammalia']
['Amphibia_image_0627.jpg', 'Amphibia_image_0214.jpg', 'Amphibia_image_0197.jpg', 'Amphibia_image_0079.jpg', 'Amphibia_image_0503.jpg', 'Amphibia_image_0110.jpg', 'Amphibia_image_0277.jpg', 'Amphibia_image_0942.jpg', 'Amphibia_image_0909.jpg', 'Amphibia_image_0024.jpg', 'Amphibia_image_0100.jpg', 'Amphibia_image_0286.jpg', 'Amphibia_image_0023.jpg', 'Amphibia_image_0884.jpg', 'Amphibia_image_0066.jpg', 'Amphibia_image_0377.jpg', 'Amphibia_image_0630.jpg', 'Amphibia_image_0775.jpg', 'Amphibia_image_0072.jpg', 'Amphibia_image_0195.jpg', 'Amphibia_image_0089.jpg', 'Amphibia_image_0829.jpg', 'Amphibia_image_0928.jpg', 'Amphibia_image_0574.jpg', 'Amphibia_image_0091.jpg', 'Amphibia_image_0179.jpg', 'Amphibia_image_0870.jpg', 'Amphibia_image_0019.jpg', 'Amphibia_image_0634.jpg', 'Amphibia_image_0783.jpg', 'Amphibia_image_0939.jpg', 'Amphibia_image_0677.jpg', 'Amphibia_image_0925.j

In [4]:
# Data transformations
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images for pretrained models
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize to match ImageNet stats
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
class AnimalPlantDataset(Dataset):
    def __init__(self, image_paths, labels=None, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')  # Open image and ensure 3 channels

        if self.transform:
            image = self.transform(image)
        
        if self.labels is not None:
            label = self.labels[idx]
            return image, label
        else:
            return image

In [6]:
from PIL import Image

# Load train images and labels
classes = os.listdir(train_dir)  # Folder names are class labels
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}  # Map class names to indices

image_paths, labels = [], []

for cls in classes:
    cls_path = os.path.join(train_dir, cls)
    for img_name in os.listdir(cls_path):
        image_paths.append(os.path.join(cls_path, img_name))
        labels.append(class_to_idx[cls])

# Split train data into training and validation
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42
)

train_dataset = AnimalPlantDataset(train_paths, train_labels, transform=train_transforms)
val_dataset = AnimalPlantDataset(val_paths, val_labels, transform=test_transforms)

# Load test data
test_paths = [os.path.join(test_dir, img_name) for img_name in os.listdir(test_dir)]
test_dataset = AnimalPlantDataset(test_paths, transform=test_transforms)

# DataLoaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load a pretrained ResNet model
model = models.resnet50(pretrained=True)

# Replace the final layer for 10 classes
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 10)
model = model.to(device)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 166MB/s]


In [8]:
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.nn import CrossEntropyLoss

criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Reduce learning rate after every 5 epochs

In [9]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    best_f1 = 0.0
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)
        
        train_acc = train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        val_preds = []
        val_targets = []
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)
                val_preds.extend(preds.cpu().numpy())
                val_targets.extend(labels.cpu().numpy())
        
        val_acc = val_correct / val_total
        
        # Calculate F1 score
        from sklearn.metrics import f1_score
        val_f1 = f1_score(val_targets, val_preds, average='weighted')
        
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")
        
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
        
        scheduler.step()

    print("Training Complete. Best F1 Score:", best_f1)


In [10]:
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10)

Epoch 1/10
Train Loss: 518.0603, Train Acc: 0.2720
Val Loss: 124.2983, Val Acc: 0.3290, Val F1: 0.3158
Epoch 2/10
Train Loss: 451.5632, Train Acc: 0.3662
Val Loss: 113.1753, Val Acc: 0.3745, Val F1: 0.3350
Epoch 3/10
Train Loss: 424.2150, Train Acc: 0.4018
Val Loss: 113.8023, Val Acc: 0.4015, Val F1: 0.3959
Epoch 4/10
Train Loss: 401.4548, Train Acc: 0.4338
Val Loss: 109.6628, Val Acc: 0.3975, Val F1: 0.3961
Epoch 5/10
Train Loss: 378.0677, Train Acc: 0.4667
Val Loss: 98.3644, Val Acc: 0.4670, Val F1: 0.4564
Epoch 6/10
Train Loss: 312.4787, Train Acc: 0.5672
Val Loss: 79.7493, Val Acc: 0.5585, Val F1: 0.5593
Epoch 7/10
Train Loss: 284.9276, Train Acc: 0.6046
Val Loss: 77.7551, Val Acc: 0.5810, Val F1: 0.5780
Epoch 8/10
Train Loss: 267.7054, Train Acc: 0.6272
Val Loss: 77.5180, Val Acc: 0.5785, Val F1: 0.5753
Epoch 9/10
Train Loss: 252.1214, Train Acc: 0.6478
Val Loss: 75.9552, Val Acc: 0.5900, Val F1: 0.5888
Epoch 10/10
Train Loss: 234.9312, Train Acc: 0.6746
Val Loss: 77.6960, Val Acc

In [11]:
# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

# Predict on test data
model.eval()
test_preds = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        test_preds.extend(preds.cpu().numpy())

# Save predictions to CSV
submission = pd.DataFrame({
    'Image_ID': [os.path.basename(p).split('.')[0] for p in test_paths],
    'Label': test_preds
})
submission.to_csv('submission.csv', index=False)

/tmp/ipykernel_17/518044222.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))
